<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)   
* [RLlib Section](#7)            

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [1]:
## install finrl library
%pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-b_5c92v3
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-b_5c92v3
  Resolved https://github.com/AI4Finance-LLC/FinRL-Library.git to commit 734f0bfe71e7b81aee37140d49ab1b1f5c6ce218
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-x2ouanlc/pyfolio_c664cd75874e48d6ae97c65019d5ddca
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-x2ouanlc/pyfolio_c664cd75874e48d6ae97c65019d5ddca
  Resolved https://github.com/quantopian/pyfolio.git to commit 4b901f6d73aa02ceb6d04b7d83502e5c6f2e81aa
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-x2ouanlc/elegantrl_a3ab9058744649a7b847ed6b364ba5c4
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-instal

  Created wheel for finrl: filename=finrl-0.3.5-py3-none-any.whl size=86457 sha256=789180e8a7fe0002b599722e22e23b371b7bf2fd4d659ac6b8f2ea082feada52
  Stored in directory: /tmp/pip-ephem-wheel-cache-uwsh9u1n/wheels/2b/0d/4a/39b40e5764855aa7c6e759ff2197a08e5d8456716a577791f8
  Created wheel for elegantrl: filename=elegantrl-0.3.3-py3-none-any.whl size=238680 sha256=0639cbc87581625fc19f9f65f0fc004e815d403663373340638f01dc68cffe4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-uwsh9u1n/wheels/a3/c3/be/03eb1f20c8650f23ab13b823d93a297a917899f5d08b04b7b9


  Created wheel for pyfolio: filename=pyfolio-0.9.2+75.g4b901f6-py3-none-any.whl size=75775 sha256=7d0370b11b6f16f986c2db7f99aee157cc9d839cc47972c6706a27ca52d1275a
  Stored in directory: /tmp/pip-ephem-wheel-cache-uwsh9u1n/wheels/da/0d/dd/aef7001cc1238aff04ec9eabfc002341f00c50deead3083855
Successfully built finrl elegantrl pyfolio
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
docker-compose 1.29.2 requires distro<2,>=1.5.0, which is not installed.
docker-compose 1.29.2 requires docker[ssh]>=5, which is not installed.
docker-compose 1.29.2 requires dockerpty<1,>=0.4.1, which is not installed.
docker-compose 1.29.2 requires docopt<1,>=0.6.1, which is not installed.
docker-compose 1.29.2 requires python-dotenv<1,>=0.13.0, which is not installed.
docker-compose 1.29.2 requires texttable<2,>=0.9.0, which is not installed.
docker-compose 1.29.2 requires PyYAML<6,>=

In [1]:
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/mnt/recoverData/linuxProgram/workspace/Finrl_python3.9/__pypackages__/3.9/lib/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
import vectorbt as vbt
import pandas as pd
start_date = '2009-01-01'
end_date = '2021-10-31'
DOW_30_TICKER = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']
data_vectorbt = vbt.YFData.download(
     DOW_30_TICKER,
     start=start_date,
     end=end_date,
     interval='1D'
 )


def convert2FinrlFormat(data):
    '''convert vectorbt format data to finrl format'''
    # dataframe schema is tuple with 7 elements: Open ,High ,Low ,Close ,Volume ,Dividends ,Stock Splits
    open = data.get()[0].stack(dropna=False).rename('open')
    high = data.get()[1].stack(dropna=False).rename('high')
    low = data.get()[2].stack(dropna=False).rename('low')
    close = data.get()[3].stack(dropna=False).rename('close')
    volume = data.get()[4].stack(dropna=False).rename('volume')
    close_rawFormat = data.get()[3]
    
    rsi_30 = vbt.RSI.run(close_rawFormat, 30).rsi.stack(dropna=False).rename(columns = {30 : 'rsi_30'})
    ma_30 = vbt.MA.run(close_rawFormat, 30).ma.stack(dropna=False).rename(columns = {30 : 'ma_30'})
    ma_60 = vbt.MA.run(close_rawFormat, 60).ma.stack(dropna=False).rename(columns = {60 : 'ma_60'})
    res = pd.concat([open,high,low,close,volume,rsi_30,ma_30,ma_60],axis=1).reset_index() # is slow because this line
    res = res.rename(columns = {'Date' : 'date', 'symbol' : 'tic'})
    res['date'] = res['date'].apply(lambda x : x.strftime('%Y-%m-%d'))
    return res

processed = convert2FinrlFormat(data_vectorbt)
processed

/mnt/recoverData/linuxProgram/workspace/Finrl_python3.9/__pypackages__/3.9/lib/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
/mnt/recoverData/linuxProgram/workspace/Finrl_python3.9/__pypackages__/3.9/lib/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
/mnt/recoverData/linuxProgram/workspace/Finrl_python3.9/__pypackages__/3.9/lib/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zo

/mnt/recoverData/linuxProgram/workspace/Finrl_python3.9/__pypackages__/3.9/lib/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
/mnt/recoverData/linuxProgram/workspace/Finrl_python3.9/__pypackages__/3.9/lib/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
/mnt/recoverData/linuxProgram/workspace/Finrl_python3.9/__pypackages__/3.9/lib/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zo

,date,tic,open,high,low,close,volume,rsi_30,ma_30,ma_60
0,2008-12-31,AXP,14.442319,15.069198,14.394098,14.908460,9625600.0,NaN,NaN,NaN
1,2008-12-31,AMGN,43.437657,44.281920,43.399628,43.924438,6287200.0,NaN,NaN,NaN
2,2008-12-31,AAPL,2.625210,2.679259,2.605972,2.606277,607541200.0,NaN,NaN,NaN
3,2008-12-31,BA,31.195798,32.290913,31.128291,32.005882,5443100.0,NaN,NaN,NaN
4,2008-12-31,CAT,29.963725,30.923660,29.963725,30.628822,6277400.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
96925,2021-10-29,V,208.105676,212.542125,207.439200,210.652161,14329800.0,44.866255,224.879986,227.021134
96926,2021-10-29,WBA,45.414664,45.821708,45.327440,45.569729,4999000.0,42.857130,46.521763,47.068580
96927,2021-10-29,WMT,146.217791,148.382738,145.871790,147.710510,7340900.0,57.362636,140.781036,143.417337
96928,2021-10-29,DIS,169.020004,170.460007,168.149994,169.070007,7598800.0,38.994183,173.359333,176.751166


In [4]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,rsi_30,ma_30,ma_60
2,2008-12-31,AAPL,2.625210,2.679259,2.605972,2.606277,607541200.0,0.000000,0.000000,0.000000
1,2008-12-31,AMGN,43.437657,44.281920,43.399628,43.924438,6287200.0,0.000000,0.000000,0.000000
0,2008-12-31,AXP,14.442319,15.069198,14.394098,14.908460,9625600.0,0.000000,0.000000,0.000000
3,2008-12-31,BA,31.195798,32.290913,31.128291,32.005882,5443100.0,0.000000,0.000000,0.000000
4,2008-12-31,CAT,29.963725,30.923660,29.963725,30.628822,6277400.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
140572,2021-10-29,UNH,449.991602,456.903744,448.654722,455.992676,2497800.0,65.073663,413.718763,412.824562
140575,2021-10-29,V,208.105676,212.542125,207.439200,210.652161,14329800.0,44.866255,224.879986,227.021134
140574,2021-10-29,VZ,51.250939,51.787852,51.163080,51.729282,17763200.0,46.768944,51.809784,52.472365
140576,2021-10-29,WBA,45.414664,45.821708,45.327440,45.569729,4999000.0,42.857130,46.521763,47.068580


In [5]:
processed_full.isnull().sum().sum()

0

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2020-07-01
## Trade data split: 2020-07-01 to 2021-10-31

In [6]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-07-01','2021-10-31')
print(len(train))
print(len(trade))

86790
10110


In [7]:
train.tail()

,date,tic,open,high,low,close,volume,rsi_30,ma_30,ma_60
2892,2020-06-30,UNH,280.551458,288.212500,279.666741,286.754181,2932900.0,51.124588,286.997206,280.002900
2892,2020-06-30,V,189.078409,191.309941,187.765157,190.737244,9040100.0,51.233292,191.485037,181.677682
2892,2020-06-30,VZ,50.184847,50.522948,49.673132,50.376743,17414800.0,48.465929,51.012122,51.464679
2892,2020-06-30,WBA,38.787097,39.210700,38.455582,39.035732,4782100.0,53.660862,39.135189,38.935129
2892,2020-06-30,WMT,115.578878,116.461082,114.919646,116.121773,6836400.0,35.064439,117.787627,119.723275


In [8]:
trade.head()

,date,tic,open,high,low,close,volume,rsi_30,ma_30,ma_60
0,2020-07-01,AAPL,90.153990,90.707079,89.855223,89.904602,110737200.0,67.961048,83.933766,77.717543
0,2020-07-01,AMGN,221.703745,241.198845,218.936211,240.153946,6575800.0,63.126934,214.858664,215.931667
0,2020-07-01,AXP,93.261315,94.935612,91.684929,92.086372,3301000.0,54.344115,97.244638,90.695525
0,2020-07-01,BA,185.880005,190.610001,180.039993,180.320007,49036700.0,59.284491,176.472335,155.614168
0,2020-07-01,CAT,123.829203,123.848334,120.479356,120.651634,2807800.0,58.750628,119.412838,113.646676


In [9]:
INDICATORS = ['rsi_30', 'ma_30','ma_60']

In [10]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 30, State Space: 151


In [11]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}
e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [12]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [13]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [14]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [15]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

/mnt/recoverData/linuxProgram/workspace/Finrl_python3.9/finrl/finrl_meta/env_stock_trading/env_stocktrading.py:158: RuntimeWarning: divide by zero encountered in double_scalars
  available_amount = self.state[0] / (self.state[index + 1]*(1 + self.buy_cost_pct[index])) # when buying stocks, we should consider the cost of trading when calculating available_amount, or we may be have cash<0
/mnt/recoverData/linuxProgram/workspace/Finrl_python3.9/finrl/finrl_meta/env_stock_trading/env_stocktrading.py:158: RuntimeWarning: invalid value encountered in double_scalars
  available_amount = self.state[0] / (self.state[index + 1]*(1 + self.buy_cost_pct[index])) # when buying stocks, we should consider the cost of trading when calculating available_amount, or we may be have cash<0


ValueError: cannot convert float NaN to integer

### Model 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

### Model 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

### Model 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

### Model 5: SAC

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=60000)

## Trading
Assume that we have $1,000,000 initial capital at 2020-07-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_risk_indicator = processed_full[(processed_full.date<'2020-07-01') & (processed_full.date>='2009-01-01')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

In [ ]:
insample_risk_indicator.turbulence.describe()

In [ ]:
insample_risk_indicator.turbulence.quantile(0.996)

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
#trade = data_split(processed_full, '2020-07-01','2021-10-31')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.tail()

In [ ]:
df_actions.head()

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])